In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import argparse
import sys
from typing import List

import cv2
import pyttsx3
from ultralytics import YOLO

In [27]:
def get_direction(x_center, frame_width):
    
    """
    Determine the horizontal direction of the detected object.
    Splits the frame into 5 zones: Far Left, Slightly Left, Center, Slightly Right, Far Right.
    """
    
    relative_pos = float(x_center / frame_width)

    print(frame_width ,x_center)
    if relative_pos < 0.2:
        return "far left"
    elif relative_pos < 0.4:
        return "slightly left"
    elif relative_pos < 0.6:
        return "center"
    elif relative_pos < 0.8:
        return "slightly right"
    else:
        return "far right"

In [5]:
def estimate_distance(pixel_width: float, known_width: float, focal_length: float) -> float:
    """Estimate distance from camera in cm."""
    if focal_length is None:
        return -1
    return (known_width * focal_length) / pixel_width

In [29]:
FOCAL_LENGTH = 707.94 
KNOWN_WIDTHS = {
    "person": 0.5,
    "cup": 0.08,
    "chair": 0.45,
    "laptop": 0.30,
    "bottle": 0.07,
}

def draw_boxes(image_bgr, results, score_threshold=0.5):
    out = image_bgr.copy()
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            if conf < score_threshold:
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cls = r.names[cls_id]
            cv2.rectangle(out, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"{cls}: {conf:.2f}"
            ((text_w, text_h), _) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            cv2.rectangle(out, (x1, y1 - text_h - 6), (x1 + text_w, y1), (0, 255, 0), -1)
            cv2.putText(out, text, (x1, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
    return out

def run_image_mode(input_path: str, score: float = 0.5, output_path: str = None, weights: str = "YOLOv10x.pt"):
    model = YOLO(weights)
    img = cv2.imread(input_path)
    if img is None:
        print(f"Error: could not read {input_path}")
        return
    results = model(img)
    drawn = draw_boxes(img, results, score)
    if output_path:
        cv2.imwrite(output_path, drawn)
        print(f"Wrote annotated image to {output_path}")
    else:
        names = extract_names(img ,results, score)
        tts(names)
        cv2.imshow('detected', drawn)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def run_webcam_mode(score: float = 0.5, frame_resize: float = 0.6, frame_skip: int = 3, weights: str = "YOLOv10x.pt"):
    model = YOLO(weights)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("ERROR: Could not open webcam")
        return
    engine = None
    last_spoken = []
    frame_idx = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_idx += 1
            if frame_idx % frame_skip != 0:
                cv2.imshow('webcam', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                continue
            if frame_resize != 1.0:
                small = cv2.resize(frame, (0, 0), fx=frame_resize, fy=frame_resize)
            else:
                small = frame
            results = model(small)
            out_small = draw_boxes(small, results, score)
            names = extract_names(small , results, score)

            
            if names and names != last_spoken:
                tts(names, engine)
                last_spoken = names
            cv2.imshow('webcam', out_small)
            cv2.waitKey(0)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except KeyboardInterrupt:
        print('Interrupted by user')
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
def tts(names: List[str], engine=None):
    if not names:
        return
    seen = set()
    unique = []
    for n in names:
        if n not in seen:
            seen.add(n)
            unique.append(n)
    sentence = "I see " + (" and ".join(unique))
    try:
        if engine is None:
            engine = pyttsx3.init()
            engine.setProperty('rate', 150)
            voices = engine.getProperty('voices')
            if len(voices) > 1:
                engine.setProperty('voice', voices[1].id)
        engine.say(sentence)
        engine.runAndWait()
    except Exception as e:
        print("TTS error:", e)
        
def extract_names(image_bgr, results, score_threshold=0.5):
    names = []
    out = image_bgr.copy()
    
    for r in results:
        for box in r.boxes:
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            label = r.names[cls_id]
            
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w = x2 - x1
            h = y2 - y1
            x_center = (x1 + x2) // 2

            if conf < score_threshold:
                continue
            cls_id = int(box.cls[0])
            distance = 0
            if label in KNOWN_WIDTHS:
                distance = estimate_distance(w , KNOWN_WIDTHS[label] , FOCAL_LENGTH)

            direction = get_direction(x_center , out.shape[1])
            if distance > 0:
                names.append(f"{r.names[cls_id]} to {direction}, about {distance:.1f} meters away")
            else:
                names.append(f"{r.names[cls_id]} to {direction}")
    return names

In [31]:
run_webcam_mode(frame_skip = 1 , frame_resize = 1)


0: 480x640 1 person, 816.4ms
Speed: 3.7ms preprocess, 816.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
640 244

0: 480x640 1 person, 888.8ms
Speed: 3.4ms preprocess, 888.8ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
640 295

0: 480x640 1 person, 851.1ms
Speed: 2.2ms preprocess, 851.1ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)
640 249

0: 480x640 1 person, 775.6ms
Speed: 2.1ms preprocess, 775.6ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
640 270

0: 480x640 1 person, 845.7ms
Speed: 2.7ms preprocess, 845.7ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
640 266
Interrupted by user


In [30]:
run_image_mode("./images/train/images/00d343d4a829121c.jpg")


0: 448x640 1 person, 5 chairs, 1 dining table, 726.0ms
Speed: 4.5ms preprocess, 726.0ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)
1024 648
1024 708
1024 424
1024 732
1024 436
1024 549


In [32]:
# Real world width of your calibration object (meters)
KNOWN_WIDTH = 0.0856  # credit card width
# Distance from camera to object during calibration (meters)
KNOWN_DISTANCE = 0.3  

cap = cv2.VideoCapture(0)

print("Press 'q' when done selecting ROI...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    cv2.imshow("Calibration - place object at known distance", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Select ROI (draw bounding box around object)
        roi = cv2.selectROI("Calibration", frame, False, False)
        x, y, w, h = roi
        pixel_width = w
        focal_length = (pixel_width * KNOWN_DISTANCE) / KNOWN_WIDTH
        print(f"Estimated focal length: {focal_length:.2f} pixels")
        break

cap.release()
cv2.destroyAllWindows()

Press 'q' when done selecting ROI...
Estimated focal length: 707.94 pixels
